<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gathering data

1.   Load the "twitter-archive-enhanced.csv" into a dataframe
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv)
3. Using tweet ids from (1) above and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest
4. Consumer API keys: 
    
    Z22dLMPEOtCL2ayqnPTbFg3sK (API key)
    
    SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj (API secret key)
    
    Access token & access token secret

    14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb (Access token)

    nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313 (Access token secret)

    Read and write (Access level)




In [0]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import requests
import io
import tweepy
import json
from google.colab import drive
import yaml
import time
import os


In [0]:
def load_yaml_file(filename: str) -> dict:
    twitter_keys = {}

    with open(filename, 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
    
    twitter_keys['consumer_key'] = cfg['twitter']['consumer']['key']
    twitter_keys['consumer_secret'] = cfg['twitter']['consumer']['secret']
    twitter_keys['access_token'] = cfg['twitter']['access']['token']
    twitter_keys['access_token_secret'] = cfg['twitter']['access']['token-secret']

    return twitter_keys

In [0]:
def print_file(file_name, lines = 10):
    with open(file_name, 'r') as f:
        for _ in range(lines):
            print (f.readline())
        

In [40]:
drive.mount('/content/gdrive')
twitter_keys_yaml_file = '/content/gdrive/My Drive/Colab Notebooks/twitter_keys.yaml'
tweet_json_file = '/content/gdrive/My Drive/Colab Notebooks/tweet_json.txt'

"""
with open(twitter_keys_yaml_file, encoding='utf-8') as file:
    line = file.readline()
    while line:
        line = file.readline()
        print(line)
"""
print(load_yaml_file(twitter_keys_yaml_file))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
{'consumer_key': 'Z22dLMPEOtCL2ayqnPTbFg3sK', 'consumer_secret': 'SuaxZay016BTIpzrYszo9Dbo0jQd38FZ8SUi0bDyy8hU6jMKdj', 'access_token': '14299634-DHvEoZI9bR2D5WOZWf82MjHX6vEnPRIosIrli6ueb', 'access_token_secret': 'nvba5wXGFrEcYKmQjztu40wsr0vG2pXnQShE2ZQv6I313'}


In [41]:
# 1. Load the file "twitter-archive-enhanced.csv" into a dataframe
twitter_archive_file_url = 'https://raw.githubusercontent.com/bthodla/danano/master/prj4/twitter-archive-enhanced.csv'

tweets_df = pd.read_csv(twitter_archive_file_url)

tweet_count = tweets_df.shape[0]
print ('%s %d' % ('Tweet Count: ', tweet_count))

Tweet Count:  2356


In [34]:
tweets_df.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [0]:
# 2. Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library

image_predictions_file_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

url_data = requests.get(image_predictions_file_url).content

img_pred_df = pd.read_csv(io.StringIO(url_data.decode('utf-8')), sep = '\t')


In [35]:
img_pred_df.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [7]:
""" 3. Using tweet ids from tweet_df and the Twitter API, load information such as "retweet count" and "favorite count" and any other interesting data items 
and store the entire set of data relating to a tweet in a JSON file called "tweet_json.txt"; then read this file line by line into a data frame and include 
tweet id, retweet count, favorite count and any other data items of interest
"""

twitter_keys = load_yaml_file(twitter_keys_yaml_file)
consumer_key = twitter_keys['consumer_key']
consumer_secret = twitter_keys['consumer_secret']
access_token = twitter_keys['access_token']
access_token_secret = twitter_keys['access_token_secret']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

error_tweets = {}
tweet_offset = 0
tweet_batch = 100
tweets_remaining = 0

# Delete the file first so that each run can add content fresh
if os.path.isfile(tweet_json_file):
    os.remove(tweet_json_file)

with open(tweet_json_file, mode = 'a', encoding = 'utf-8') as f:
    while tweet_batch:
        tweet_ids = tweets_df[tweet_offset:tweet_offset + tweet_batch]['tweet_id']
        for tweet_id in tweet_ids:
            try:
                status = api.get_status(tweet_id)
                f.write(json.dumps(status._json) + '\n')

            except tweepy.TweepError as e:
                error_tweets[status.id] = e
            
        tweets_remaining = tweet_count - tweet_offset
        tweet_batch = tweet_batch if tweet_batch < tweets_remaining else tweets_remaining
        tweet_offset += tweet_batch
        # print ('%s %d %s %d' % ('Tweets Remaining: ', tweets_remaining, 'Tweet Offset: ', tweet_offset))
        # time.sleep(1 * 5)


Rate limit reached. Sleeping for: 771
Rate limit reached. Sleeping for: 772


In [8]:
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    print(len(f.readlines()))

2339


In [0]:
# Read "tweet_json.txt" file line by line into a data frame and include tweet id, retweet count, favorite count and any other data items of interest

columns = ['tweet_id', 'text', 'retweet_count', 'favorite_count', 'retweeted', 'favorited', 'media_url']
tweets_mini_df = pd.DataFrame(columns = columns)

index = 0
with open(tweet_json_file, mode = 'r', encoding = 'utf-8') as f:
    for tweet in f:
        tweet_json = json.loads(tweet)
        tweet_id = tweet_json['id']
        text = tweet_json['text']
        retweet_count = tweet_json['retweet_count']
        favorite_count = tweet_json['favorite_count']
        retweeted = tweet_json['retweeted']
        favorited = tweet_json['favorited']
        try:
            media_url = tweet_json['entities']['media'][0]['media_url_https']
        except KeyError as e:
            media_url = ''

        tweets_mini_df.loc[index] = pd.Series({'tweet_id': tweet_id, 
                                        'text': text, 
                                        'retweet_count': retweet_count, 
                                        'favorite_count': favorite_count,
                                        'retweeted': retweeted,
                                        'favorited': favorited,
                                        'media_url': media_url})
        index +=1


In [32]:
tweets_mini_df.head()



,tweet_id,text,retweet_count,favorite_count,retweeted,favorited,media_url
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,8226,37748,False,False,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg
1,892177421306343426,This is Tilly. She's just checking pup on you....,6079,32430,False,False,
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,4023,24427,False,False,
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,8379,41085,False,False,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg
4,891327558926688256,This is Franklin. He would like you to stop ca...,9082,39281,False,False,


### Assessing data

Key Points

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.

* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.

* Cleaning includes merging individual pieces of data according to the rules of tidy data.

* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.

* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.


In [50]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

#### twitter-archive-enhanced.csv

In [61]:
# There are 181 rows in this dataset that are retweets. This is based on the fact that the column "retweeted_status_id" is not null. 
# This column points to the original tweet which this has retweeted. These need to be removed from the dataset as we only want original ratings

tweets_df[tweets_df['retweeted_status_id'].notna()].shape[0]

181

In [51]:
# "timestamp" is represented as a string and should be converted to datetime
tweets_df['timestamp'].head(5)

0    2017-08-01 16:23:56 +0000
1    2017-08-01 00:17:27 +0000
2    2017-07-31 00:18:03 +0000
3    2017-07-30 15:58:51 +0000
4    2017-07-29 16:00:24 +0000
Name: timestamp, dtype: object

In [87]:
# Dog stages ('doggo', 'floofer', 'pupper', 'puppo') are not fully recorded in the respective columns with most values having been set to "None"
# This needs to be corrected

tweets_df[tweets_df['text'].str.contains("pupper")][['tweet_id', 'pupper', 'text']]

,tweet_id,pupper,text
29,886366144734445568,pupper,This is Roscoe. Another pupper fallen victim t...
49,882762694511734784,pupper,This is Gus. He's quite the cheeky pupper. Alr...
54,881666595344535552,None,This is Gary. He couldn't miss this puppertuni...
56,881536004380872706,pupper,Here is a pupper approaching maximum borkdrive...
78,877611172832227328,pupper,RT @rachel2195: @dog_rates the boyfriend and h...
82,876838120628539392,pupper,This is Ginger. She's having a ruff Monday. To...
92,874296783580663808,pupper,This is Jed. He may be the fanciest pupper in ...
97,873337748698140672,pupper,RT @dog_rates: This is Sierra. She's one preci...
98,873213775632977920,pupper,This is Sierra. She's one precious pupper. Abs...
107,871762521631449091,pupper,This is Rover. As part of pupper protocol he h...
